# Import Dependencies

In [75]:
# Import and Dependencies
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import io
import os
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib_venn import venn3, venn2
import sqlite3 as sql
import seaborn as sns
from pathlib import Path
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
ohe = OneHotEncoder()
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# Colab Featured Interactive Table
from google.colab import data_table 
data_table.enable_dataframe_formatter()

# Load Data from CSV

In [76]:
# File Import from PC, CSV Reader
#from google.colab import files
#uploaded = files.upload()
#lung_cancer_df = pd.read_csv(io.BytesIO(uploaded['survey_lung_cancer.csv']))

# Load data from DataBase

In [77]:
try:
  conn = sql.connect('Cancer.db')
  lung_cancer_df.to_sql('lung_cancer', conn)
except:
  print("Database already created")

Database already created


In [78]:
# Call data from DataBase to DataFrame
conn = sql.connect('Cancer.db')
lung_cancer_data = pd.read_sql('SELECT * FROM lung_cancer', conn)

In [79]:
# Examine DataFrame
lung_cancer_data

,index,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,2,2,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,4,2,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,305,1,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,306,1,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,307,1,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [80]:
# Examine DataFrame
lung_cancer_data

,index,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,2,2,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,4,2,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,305,1,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,306,1,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,307,1,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [81]:
lung_cancer_data.columns

Index(['index', 'GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER'],
      dtype='object')

In [82]:
# Drop unnecessary columns
#lung_cancer_df=lung_cancer_df.drop(['PEER_PRESSURE', 'ALCOHOL CONSUMING', 'ALLERGY '], axis=1)

# Drop any rows with null values
lung_cancer_data = lung_cancer_data.dropna()

# Change Yes and No to 1 and 0
key_rev = {'YES' : 2, 'NO' : 1}
lung_cancer_data = lung_cancer_data.replace(key_rev)

# Change Male to 1 and Female to 2
key_rev_gender = {'M' : 1, 'F' : 2}
lung_cancer_data = lung_cancer_data.replace(key_rev_gender)

lung_cancer_data

,index,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,2,2,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,4,2,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,305,1,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,306,1,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,307,1,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [83]:
lung_cancer_data["LUNG_CANCER"].value_counts()

1    270
0     39
Name: LUNG_CANCER, dtype: int64

# Split the Data into Training and Testing

In [84]:
# Create our Feature
X = lung_cancer_data.drop("LUNG_CANCER", axis=1)
X = pd.get_dummies(X)

# Create our Target
y = lung_cancer_data["LUNG_CANCER"]

In [85]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)
print(f'Train set lenght: {X_train.count()} records')
print(f'Test set lenght: {X_test.count()} records')

Train set lenght: index                    207
GENDER                   207
AGE                      207
SMOKING                  207
YELLOW_FINGERS           207
ANXIETY                  207
PEER_PRESSURE            207
CHRONIC DISEASE          207
FATIGUE                  207
ALLERGY                  207
WHEEZING                 207
ALCOHOL CONSUMING        207
COUGHING                 207
SHORTNESS OF BREATH      207
SWALLOWING DIFFICULTY    207
CHEST PAIN               207
dtype: int64 records
Test set lenght: index                    102
GENDER                   102
AGE                      102
SMOKING                  102
YELLOW_FINGERS           102
ANXIETY                  102
PEER_PRESSURE            102
CHRONIC DISEASE          102
FATIGUE                  102
ALLERGY                  102
WHEEZING                 102
ALCOHOL CONSUMING        102
COUGHING                 102
SHORTNESS OF BREATH      102
SWALLOWING DIFFICULTY    102
CHEST PAIN               102
dtype: int64 rec

In [86]:
# Check the Feature values
X.describe()

,index,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,154.000000,1.475728,62.673139,1.563107,1.569579,1.498382,1.501618,1.504854,1.673139,1.556634,1.556634,1.556634,1.579288,1.640777,1.469256,1.556634
std,89.344838,0.500221,8.210301,0.496806,0.495938,0.500808,0.500808,0.500787,0.469827,0.497588,0.497588,0.497588,0.494474,0.480551,0.499863,0.497588
min,0.000000,1.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,77.000000,1.000000,57.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,154.000000,1.000000,62.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000
75%,231.000000,2.000000,69.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,308.000000,2.000000,87.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [87]:
# Check the balance of the Target values
y.value_counts()

1    270
0     39
Name: LUNG_CANCER, dtype: int64

In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

len(X_train), len(X_test)

(231, 78)

### Scale the Data

In [89]:
# Scale the Data
scaled_data = StandardScaler()
X_train = scaled_data.fit_transform(X_train)
X_test = scaled_data.transform(X_test)

### Balance the Data

In [90]:
# Balance the data since the y values are imblanced
over_samp =  RandomOverSampler(random_state=42)
X_train_res, y_train_res = over_samp.fit_resample(X, y)
X_train_res.shape, y_train_res.shape

((540, 16), (540,))

# Linear Regression Model with all Features

In [91]:
# Build Linear Regression Model with all the features
# Create and fit the model
model = LinearRegression()
model.fit(X_test, y_test)

LinearRegression()

In [92]:
LinearRegressionScore = model.score(X_test, y_test)
print("Accuracy Score for Linear Regression model including all features:",LinearRegressionScore*100,"%")

Accuracy Score for Linear Regression model including all features: 33.579933843168575 %


# Linear Regression Model of Behavioral Features

In [93]:
lung_cancer_data

,index,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,2,2,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,4,2,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,305,1,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,306,1,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,307,1,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [94]:
# Pick out the columns needed for the Behavioral Features
behavioral_features = ['SMOKING', 'PEER_PRESSURE', 'ALCOHOL CONSUMING']

In [95]:
# Create a DataFrame for the Behavioral Features
behavioral_features_df = lung_cancer_data[np.append(behavioral_features,'LUNG_CANCER')].copy()
behavioral_features_df

,SMOKING,PEER_PRESSURE,ALCOHOL CONSUMING,LUNG_CANCER
0,1,1,2,1
1,2,1,1,1
2,1,2,1,0
3,2,1,2,0
4,1,1,1,0
...,...,...,...,...
304,1,2,2,1
305,2,1,2,1
306,2,1,2,1
307,2,1,2,1


In [96]:
# Create our Feature
X = behavioral_features_df.drop("LUNG_CANCER", axis=1)
X = pd.get_dummies(X)

# Create our Target
y = behavioral_features_df["LUNG_CANCER"]

In [97]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)
print(f'Train set lenght: {X_train.count()} records')
print(f'Test set lenght: {X_test.count()} records')

Train set lenght: SMOKING              207
PEER_PRESSURE        207
ALCOHOL CONSUMING    207
dtype: int64 records
Test set lenght: SMOKING              102
PEER_PRESSURE        102
ALCOHOL CONSUMING    102
dtype: int64 records


In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

len(X_train), len(X_test)

(231, 78)

In [99]:
# Build Linear Regression Model with all the features
# Create and fit the model
model = LinearRegression()
model.fit(X_test, y_test)

LinearRegression()

In [100]:
# Print the Accuracy Score for the Behavioral Features Linear Regression Model
LinearRegressionScore = model.score(X_test, y_test)
print("Accuracy Score for Linear Regression model for Behavioral features:",LinearRegressionScore*100,"%")

Accuracy Score for Linear Regression model for Behavioral features: 14.160790142668212 %


# Linear Regression for Symptomatic Features

In [101]:
lung_cancer_data

,index,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0,1,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,1,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,2,2,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,3,1,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,4,2,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,305,1,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,306,1,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,307,1,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [102]:
# Pick out the columns needed for the Symptomatic Features
symptomatic_features = ['YELLOW_FINGERS', 'ANXIETY', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING', 'COUGHING', 'SHORTNESS OF BREATH', 'SWALLOWING DIFFICULTY', 'CHEST PAIN']

In [103]:
# Create a a DataFrame for the Symptomatic Features
symptomatic_features_df = lung_cancer_data[np.append(symptomatic_features,'LUNG_CANCER')].copy()
symptomatic_features_df

,YELLOW_FINGERS,ANXIETY,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,2,2,1,2,1,2,2,2,2,2,1
1,1,1,2,2,2,1,1,2,2,2,1
2,1,1,1,2,1,2,2,2,1,2,0
3,2,2,1,1,1,1,1,1,2,2,0
4,2,1,1,1,1,2,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
304,1,1,2,2,1,1,2,2,2,1,1
305,1,1,1,2,2,2,2,2,1,2,1
306,1,1,1,1,2,2,2,1,1,2,1
307,1,2,1,2,2,1,2,2,1,2,1


In [104]:
# Create our Feature
X = symptomatic_features_df.drop("LUNG_CANCER", axis=1)
X = pd.get_dummies(X)

# Create our Target
y = symptomatic_features_df["LUNG_CANCER"]

In [105]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)
print(f'Train set lenght: {X_train.count()} records')
print(f'Test set lenght: {X_test.count()} records')

Train set lenght: YELLOW_FINGERS           207
ANXIETY                  207
CHRONIC DISEASE          207
FATIGUE                  207
ALLERGY                  207
WHEEZING                 207
COUGHING                 207
SHORTNESS OF BREATH      207
SWALLOWING DIFFICULTY    207
CHEST PAIN               207
dtype: int64 records
Test set lenght: YELLOW_FINGERS           102
ANXIETY                  102
CHRONIC DISEASE          102
FATIGUE                  102
ALLERGY                  102
WHEEZING                 102
COUGHING                 102
SHORTNESS OF BREATH      102
SWALLOWING DIFFICULTY    102
CHEST PAIN               102
dtype: int64 records


In [106]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

len(X_train), len(X_test)

(231, 78)

In [107]:
# Build Linear Regression Model with all the features
# Create and fit the model
model = LinearRegression()
model.fit(X_test, y_test)

LinearRegression()

In [108]:
LinearRegressionScore = model.score(X_test, y_test)
print("Accuracy Score for Linear Regression model for the Symptomatic features:",LinearRegressionScore*100,"%")

Accuracy Score for Linear Regression model for the Symptomatic features: 27.196527125748617 %
